# llama-index 练习


## 简单 RAG 查询


In [10]:
import chromadb
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    get_response_synthesizer,
    Settings,
)
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core.node_parser import SentenceSplitter
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core import StorageContext
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core.query_engine import RetrieverQueryEngine

In [11]:
# 设置嵌入模型和语言模型
Settings.embed_model = OllamaEmbedding(model_name="yxl/m3e:latest")
Settings.llm = Ollama(model="qwen2.5:0.5b", request_timeout=360)

In [12]:
# 初始化 Chroma 客户端，指定数据存储路径为当前目录下的 chroma_db 文件夹
db = chromadb.PersistentClient(path="./chroma_db")

# 获取或创建名为 "quickstart" 的集合，如果该集合不存在，则创建它
chroma_collection = db.get_or_create_collection("quickstart")

# 使用上述集合创建一个 ChromaVectorStore 实例，以便 llama_index 可以与 Chroma 集合进行交互
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)

# 创建一个存储上下文，指定向量存储为刚刚创建的 ChromaVectorStore 实例
storage_context = StorageContext.from_defaults(vector_store=vector_store)

In [16]:
# 读取文档
documents = SimpleDirectoryReader("C:/Users/Admin/Desktop/Data/").load_data()

# 构建索引
index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context,
    transformations=[SentenceSplitter(chunk_size=256)],
)

In [14]:
# 配置检索器
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=5,  # 返回最相似的前 n 个文档片段
)

# 配置响应合成器
response_synthesizer = get_response_synthesizer()

# 组装查询引擎
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

In [15]:
# 执行查询
response = query_engine.query("数字经济")
print(response)  # 输出查询结果

数字经济是一门新兴的经济形态，通过将物理世界的信息、过程和服务转换成数字形式，以提高生产效率、优化资源配置和促进创新。
